###import packages

In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np

#import matplotlib.pyplot as plt
import torch
import numpy as np

import albumentations as albu
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import cv2
from sklearn.metrics import accuracy_score , f1_score , jaccard_score ,precision_score , recall_score
from glob import glob
from tqdm import tqdm
import pandas as pd
import segmentation_models_pytorch as smp

## Loading data

In [2]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    
    CLASSES = ['sky', 'building', 'pole', 'road', 'pavement', 
               'tree', 'signsymbol', 'fence', 'car', 
               'pedestrian', 'bicyclist', 'unlabelled']
    """
    CLASSES = ['masks']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image  = cv2.resize(image,(256,256))
        mask = cv2.imread(self.masks_fps[i], 0)
        mask  = cv2.resize(mask,(256,256))
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [3]:

#import zipfile
#with zipfile.ZipFile('/content/drive/MyDrive/mask_segmentation/dataset/DatasetV4.zip', 'r') as zip_ref:
 #   zip_ref.extractall('/content/')

In [4]:
DATA_DIR = 'E:\drive D\python code\Boundary-Aware Segmentation Network for Mobile and Web Applications\code\BASNet\OtherNetwork\Dataset_Test\\'

x_test_dir = os.path.join(DATA_DIR, 'image/')
y_test_dir = os.path.join(DATA_DIR, 'mask/')

### Dataloader

Writing helper class for data extraction, tranformation and preprocessing  
https://pytorch.org/docs/stable/data

In [5]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    
    CLASSES = ['sky', 'building', 'pole', 'road', 'pavement', 
               'tree', 'signsymbol', 'fence', 'car', 
               'pedestrian', 'bicyclist', 'unlabelled']
    """
    CLASSES = ['masks']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image  = cv2.resize(image,(256,256))
        mask = cv2.imread(self.masks_fps[i], 0)
        mask  = cv2.resize(mask,(256,256))
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask ,self.images_fps[i]
        #return self.images_fps[i]  
    def __len__(self):
        return len(self.ids)

### Augmentations


In [153]:
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.Precision(),]

In [154]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        #albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        #albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        #albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.GaussNoise(p=0.2),
        #albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=0.5),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=0.4),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(256, 256)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    print(albu.__version__)
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

## Test best saved model

In [187]:
#best_model = torch.load('D:\python code\paper code\model\model\Morphological\weight\\deeplabv3++\\best_model-22-.pth',map_location=torch.device('cuda'))
best_model = torch.load('E:\drive D\\Net\\Unet++\\efficient-b7\\best_model_efficientnet-b7_29-.pth',map_location=torch.device('cuda'))

In [188]:
# create test dataset
CLASSES = ['masks']

test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_dataloader = DataLoader(test_dataset)

1.0.3


In [189]:
# # evaluate model on test set
# test_epoch = smp.utils.train.ValidEpoch(
#     model=best_model,
#     loss=smp.utils.losses.DiceLoss(),
#     metrics=metrics,
#     device='cuda',
# )
# logs = test_epoch.run(test_dataloader)

In [190]:
def apply_dilated(img,kernelsize ,iteration):
    kernel = np.ones(kernelsize,np.uint8)
    result_img = cv2.dilate(img,kernel ,iterations=iteration)
    return result_img

def apply_morph_opening(img,kernelSize):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return opening

def apply_erode(img,kernelsize ,iteration):
    kernel = np.ones(kernelsize,np.uint8)
    result_img = cv2.erode(img.copy(),kernel ,iterations=iteration)
    return result_img

def apply_morph_closing(img,kernelSize):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return closing

def write_image(dir_net,pr_mask):
    cv2.imwrite('E:\drive D\\Net\\Unet++\\efficient-b7\\'+dir_net+'\\'+filename,pr_mask)



In [191]:
len(test_dataset)

274

In [192]:
DEVICE = 'cuda'
for i in range(len(test_dataset)):
    #n = np.random.choice(len(test_dataset))
    
    #image_vis = test_dataset_vis[n][0].astype('uint8')
    image, gt_mask,name = test_dataset[i]
    
    gt_mask = gt_mask.squeeze()
    
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    pr_mask = pr_mask.reshape(256,256,1)
    pr_mask = pr_mask[:,:,0]
    pr_mask = cv2.resize(pr_mask,(256,256))
    pr_mask *=255
    ret , pr_mask = cv2.threshold(pr_mask, 120, 255, cv2.THRESH_BINARY_INV)

    # pr_mask_erode = apply_erode(pr_mask,(3,3),2)
    # pr_mask_dilated = apply_dilated(pr_mask , (3,3),2)
    #
    # pr_mask_dilated1 = apply_dilated(pr_mask , (7,7),1)
    # pr_mask_dilated_erode = apply_erode(pr_mask_dilated1,(3,3),4)
    # pr_mask_dilated_erode1 = apply_erode(pr_mask_dilated1,(3,3),5)
    filename = os.path.basename(os.path.normpath(name))
    write_image('result',pr_mask)

In [203]:
def accuracy_f1score_cal(ground_truth_path , predicted_path,csvName):
    ground_truth_label = sorted(glob(ground_truth_path)) 
    predict_label = sorted(glob(predicted_path))

    SCORE=[]
    for truth,predict in tqdm(zip(ground_truth_label,predict_label) , total=len(ground_truth_label)):
        image_name = truth.split("/")[-1]
        truth = cv2.imread(truth)
        truth = cv2.resize(truth,(256,256))
        truth = truth/255
        truth = truth.astype(np.int32)          
        #cv2.imshow('d',truth)
        #cv2.waitKey(0)
        predict = cv2.imread(predict)
        predict = predict/255 
        predict = predict.astype(np.int32)  
        truth = truth.flatten()
        predict = predict.flatten()

        acc_value = accuracy_score(truth,predict)
        f1 = f1_score(truth,predict , labels=[0,1],average="binary")
        jaccard = jaccard_score(truth,predict , labels=[0,1],average="binary")
        preceision = precision_score(truth,predict , labels=[0,1],average="binary")
        recall = recall_score(truth,predict , labels=[0,1],average="binary")

        SCORE.append([image_name , acc_value,f1 ,jaccard , preceision ,recall])
    score = [ s[1:] for s in SCORE]
    score = np.mean(score , axis=0)
    df = pd.DataFrame(SCORE , columns=["imagename" , "accuracy" , "F1" , "jaccard"  ,"preceision" , "recall"])

    df.to_csv(csvName+".csv")
    return score

In [204]:
nets = ["Deep", "Deep++","FPN", "Linknet","Manet","Pan","Psp","Unet","Unet++"]
backBone = ["efficient-b7" , "resnet50" , "vgg19"]
rootDir = "E:\drive D\\Net\\"
for i in nets:
    for ii in backBone:
        resultDir = os.path.join(rootDir,i,ii,"result")
        if len(os.listdir(resultDir)) > 0:
            acc_erode = accuracy_f1score_cal("Dataset_Test/mask/*",resultDir+"/*",csvName= i+"_"+ii)
            print(f"accuracy_result: {acc_erode[0]:0.5f}")
            print(f"f1_Score_result: {acc_erode[1]:0.5f}")
            print(f"IoU:result {acc_erode[2]:0.5f}")
            print(f"pre:result {acc_erode[3]:0.5f}")
            print(f"recall:result {acc_erode[4]:0.5f}")
            print("///////////////////////////////////////////////////////////////")

100%|██████████| 274/274 [00:55<00:00,  4.97it/s]


accuracy_result: 0.95889
f1_Score_result: 0.93960
IoU:result 0.88912
pre:result 0.91190
recall:result 0.97234
///////////////////////////////////////////////////////////////


100%|██████████| 274/274 [00:55<00:00,  4.95it/s]


accuracy_result: 0.81284
f1_Score_result: 0.73779
IoU:result 0.60151
pre:result 0.71406
recall:result 0.80654
///////////////////////////////////////////////////////////////


100%|██████████| 274/274 [00:54<00:00,  5.00it/s]


...

accuracy_result: 0.96129
f1_Score_result: 0.94336
IoU:result 0.89524
pre:result 0.91565
recall:result 0.97567
///////////////////////////////////////////////////////////////


100%|██████████| 274/274 [00:53<00:00,  5.11it/s]


accuracy_result: 0.95678
f1_Score_result: 0.93750
IoU:result 0.88526
pre:result 0.90952
recall:result 0.97106
///////////////////////////////////////////////////////////////


100%|██████████| 274/274 [00:53<00:00,  5.08it/s]

accuracy_result: 0.95998
f1_Score_result: 0.94195
IoU:result 0.89242
pre:result 0.91602
recall:result 0.97225
///////////////////////////////////////////////////////////////


In [195]:

acc_erode = accuracy_f1score_cal("Dataset_Test/mask/*","E:\drive D\\Net\\Unet++\\efficient-b7\\result/*" )
print(f"accuracy_result: {acc_erode[0]:0.5f}")
print(f"f1_Score_result: {acc_erode[1]:0.5f}")
print(f"IoU:result {acc_erode[2]:0.5f}")
print(f"pre:result {acc_erode[3]:0.5f}")
print(f"recall:result {acc_erode[4]:0.5f}")
print("///////////////////////////////////////////////////////////////")

100%|██████████| 274/274 [00:53<00:00,  5.11it/s]

accuracy_result: 0.96129
f1_Score_result: 0.94336
IoU:result 0.89524
pre:result 0.91565
recall:result 0.97567


In [196]:

acc_erode = accuracy_f1score_cal("../Morphological/DatasetV4/DatasetV4/test_masks1/test/*","../Morphological/weight/result/erode/*")
print(f"accuracy_erode: {acc_erode[0]:0.5f}")
print(f"f1_Score_erode: {acc_erode[1]:0.5f}")
print(f"IoU_erode: {acc_erode[2]:0.5f}")
print(f"pre_erode: {acc_erode[3]:0.5f}")
print(f"recall_erode: {acc_erode[4]:0.5f}")



0it [00:00, ?it/s]
C:\Users\sadjad\anaconda3\envs\mytfenv\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\sadjad\anaconda3\envs\mytfenv\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: invalid index to scalar variable.

In [43]:
acc_dilated = accuracy_f1score_cal("../Morphological/DatasetV4/DatasetV4/test_masks1/test/*","../Morphological/weight/result/dilated/*")
print(f"accuracy_dilated:{acc_dilated[0]:0.5f}")
print(f"f1_Score_dilated:{acc_dilated[1]:0.5f}")
print(f"IoU_dilated:{acc_dilated[2]:0.5f}")
print(f"pre_dilated:{acc_dilated[3]:0.5f}")
print(f"recall_dilated:{acc_dilated[4]:0.5f}")

100%|██████████| 21/21 [00:04<00:00,  4.83it/s]

accuracy_dilated:0.97505
f1_Score_dilated:0.93117
IoU_dilated:0.87488
pre_dilated:0.89974
recall_dilated:0.97102


In [44]:
acc_dilated1 = accuracy_f1score_cal("../Morphological/DatasetV4/DatasetV4/test_masks1/test/*","../Morphological/weight/result/dilated1/*")
print(f"accuracy_dilated_opening:{acc_dilated1[0]:0.5f}")
print(f"f1_Score_dilated_opening:{acc_dilated1[1]:0.5f}")
print(f"IoU_dilated_opening:{acc_dilated1[2]:0.5f}")
print(f"pre_dilated_opening:{acc_dilated1[3]:0.5f}")
print(f"recall_dilated_opening:{acc_dilated1[4]:0.5f}")

100%|██████████| 21/21 [00:04<00:00,  4.85it/s]

accuracy_dilated_opening:0.98052
f1_Score_dilated_opening:0.94717
IoU_dilated_opening:0.90434
pre_dilated_opening:0.93527
recall_dilated_opening:0.96550


In [ ]:
no morph                         
accuracy:0.95996
f1_Score:0.94198            
IoU:0.89187
pre:0.91148
recall:0.97645

erode(10,10),25
accuracy:0.96665
f1_Score:0.95008
IoU:0.90639
pre:0.95387
recall:0.94780

erode(7,7),25
accuracy:0.96665
f1_Score:0.95008
IoU:0.90639
pre:0.95387
recall:0.94780

erode(7,7),45
accuracy:0.96665
f1_Score:0.95008
IoU:0.90639
pre:0.95387
recall:0.94780

erode(7,7),45 opening (4,4)
accuracy:0.96511
f1_Score:0.94753
IoU:0.90181
pre:0.95438
recall:0.94223



erode 
(7,7),1
accuracy_erode:0.96665
f1_Score_erode:0.95008
IoU_erode:0.90639
pre_erode:0.95387
recall_erode:0.94780

(7,7),1
accuracy_erode: 0.96665
f1_Score_erode: 0.95008
IoU_erode: 0.90639
pre_erode: 0.95387
recall_erode: 0.94780

(4,4),1
accuracy_erode: 0.96520
f1_Score_erode: 0.94866
IoU_erode: 0.90382
pre_erode: 0.93526
recall_erode: 0.96403

(3,3),2
accuracy_erode: 0.96653
f1_Score_erode: 0.95044
IoU_erode: 0.90703
pre_erode: 0.94177
recall_erode: 0.96080

(8,8),1
accuracy_erode: 0.96533
f1_Score_erode: 0.94764
IoU_erode: 0.90203
pre_erode: 0.95975
recall_erode: 0.93730

(6,6),1
accuracy_erode: 0.96619
f1_Score_erode: 0.94958
IoU_erode: 0.90548
pre_erode: 0.94816
recall_erode: 0.95249



dilated
(7,7),45
accuracy_dilated:0.93916
f1_Score_dilated:0.91551
IoU_dilated:0.84623
pre_dilated:0.85229
recall_dilated:0.99177

(10,10),10
accuracy_dilated:0.93916
f1_Score_dilated:0.91551
IoU_dilated:0.84623
pre_dilated:0.85229
recall_dilated:0.99177

(4,4)
accuracy_dilated:0.95159
f1_Score_dilated:0.93107
IoU_dilated:0.87279
pre_dilated:0.88443
recall_dilated:0.98519

(3,3)
accuracy_dilated:0.95454
f1_Score_dilated:0.93489
IoU_dilated:0.87940
pre_dilated:0.89364
recall_dilated:0.98224

(2,2)
accuracy_dilated:0.95773
f1_Score_dilated:0.93903
IoU_dilated:0.88669
pre_dilated:0.90351
recall_dilated:0.97947


opening
(4,4)
accuracy_opening:0.96024
f1_Score_opening:0.94220
IoU_opening:0.89229
pre_opening:0.91276
recall_opening:0.97547

(20,20)
accuracy_opening:0.96168
f1_Score_opening:0.94434
IoU_opening:0.89599
pre_opening:0.91919
recall_opening:0.97264

(3,3)
accuracy_opening:0.95997
f1_Score_opening:0.94201
IoU_opening:0.89192
pre_opening:0.91149
recall_opening:0.97649




closing
(7,7)
accuracy_closing:0.95921
f1_Score_closing:0.94111
IoU_closing:0.89032
pre_closingg:0.90798
recall_closing:0.97861

(4,4)
accuracy_closing:0.96009
f1_Score_closing:0.94201
IoU_closing:0.89196
pre_closingg:0.91178
recall_closing:0.97620

(3,3)
accuracy_closing:0.95998
f1_Score_closing:0.94203
IoU_closing:0.89196
pre_closingg:0.91084
recall_closing:0.97728

accuracy_closing:0.96021
f1_Score_closing:0.94215
IoU_closing:0.89221
pre_closingg:0.91263
recall_closing:0.97551

dilated_opening
(7,7),45 (4,4)
accuracy_dilated_opening:0.93967
f1_Score_dilated_opening:0.91591
IoU_dilated_opening:0.84700
pre_dilated_opening:0.85399
recall_dilated_opening:0.99054

(4,4),45 (4,4)
accuracy_dilated_opening:0.95117
f1_Score_dilated_opening:0.93040
IoU_dilated_opening:0.87167
pre_dilated_opening:0.88380
recall_dilated_opening:0.98460

(3,3),45 (3,3)
accuracy_dilated_opening:0.95454
f1_Score_dilated_opening:0.93489
IoU_dilated_opening:0.87940
pre_dilated_opening:0.89364
recall_dilated_opening:0.98224

(8,8),45 (2,2)
accuracy_dilated_opening:0.96282
f1_Score_dilated_opening:0.94357
IoU_dilated_opening:0.89475
pre_dilated_opening:0.95939
recall_dilated_opening:0.92972

(6,6),100 (2,2)
accuracy_dilated_opening:0.96408
f1_Score_dilated_opening:0.94616
IoU_dilated_opening:0.89937
pre_dilated_opening:0.94795
recall_dilated_opening:0.94585

(7,7),100 (2,2)
accuracy_dilated_opening:0.96507
f1_Score_dilated_opening:0.94746
IoU_dilated_opening:0.90169
pre_dilated_opening:0.95422
recall_dilated_opening:0.94224